In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
# Load data
data746 = pd.read_csv('../Datasets/HIV-1 Protease/746Data.txt', sep=',', header=None)
data1625 = pd.read_csv('../Datasets/HIV-1 Protease/1625Data.txt', sep=',', header=None)

In [3]:
array = []
for i in range(0, len(data746)):
    array.append(list(data746[0][i])+[data746[1][i]])

In [4]:
for i in range(0, len(data1625)):
    array.append(list(data1625[0][i])+[data1625[1][i]])

In [5]:
dataset = pd.DataFrame(array)

In [6]:
dataset.replace({'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}, inplace=True)

In [7]:
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [8]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train.values, y_train)
y_hat = automl.predict(X_test.values)

In [9]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.9612141652613828

In [10]:
import pickle
with open('./models/hiv_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [9]:
import pickle
with open('./models/hiv_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

len(automl.show_models())

15

# LIME

In [10]:
import lime
import lime.lime_tabular
import tqdm 

In [11]:
y_test.unique()

array([ 1, -1])

In [12]:
categorical_features = X_train.columns.tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['Not Cleaved', 'Cleaved'], categorical_features=categorical_features, discretize_continuous=True)

In [13]:
text_x = X_test.values

In [14]:
exp_fn = lambda i: explainer.explain_instance(X_test.iloc[i], automl.predict_proba, num_features=len(X_test.columns))
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [15]:
import metrics

In [17]:
exp1 = exp_fn_wrap(text_x)
exp2 = exp_fn_wrap(text_x)

100%|██████████| 593/593 [08:43<00:00,  1.13it/s]


In [18]:
np.save('./explanations/hiv1.npy', exp1)
np.save('./explanations/hiv2.npy', exp2)

In [32]:
exp1 = np.load('./explanations/hiv1.npy')
exp2 = np.load('./explanations/hiv2.npy')

In [33]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [34]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(X_test.columns))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [35]:
i, s, sb

((100.0, 0, 593), (0, 593, 351649, 0.0), (79, 593))

In [36]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [37]:
sim

0.12303446352589929

In [38]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(text_x))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(automl, text_x[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 593/593 [23:38<00:00,  2.39s/it]


In [39]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.06996966139320523
0.0
2.224283305227656
0.07142857142857144
0.0
0.0


In [40]:
trust = metrics.calc_trust_score(automl, text_x, exp1, 3, X_train.columns.tolist())

100%|██████████| 593/593 [1:08:17<00:00,  6.91s/it]


In [41]:
trust

0.7403035413153457

# CIU

In [19]:
from ciu import determine_ciu
import tqdm
import metrics

In [20]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [21]:
feat_list = X_train.columns.tolist()

In [22]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], automl.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [23]:
exp1 = exp_fn_blk(X_test)
exp2 = exp_fn_blk(X_test)

100%|██████████| 593/593 [14:07<00:00,  1.43s/it]


In [24]:
np.save('./explanations/hiv_ciu1.npy', exp1)
np.save('./explanations/hiv_ciu2.npy', exp2)

In [13]:
exp1 = np.load('./explanations/hiv_ciu1.npy')
exp2 = np.load('./explanations/hiv_ciu2.npy')

In [25]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [26]:
i, s, sb

((40.640809443507585, 352, 593),
 (44, 593, 351649, 0.01251247693012066),
 (9, 593))

In [27]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [28]:
sim

0.773439292139021

In [29]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(automl, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 593/593 [22:24<00:00,  2.27s/it]


In [30]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

-0.08278421170477931
0.0
2.1551433389544687
-0.1666666666666667
0.0
0.0


In [31]:
metrics.calc_trust_score(automl, X_test.to_numpy(), exp1, 3, X_train.columns.to_list())

100%|██████████| 593/593 [1:12:55<00:00,  7.38s/it]


0.7363687464867903

# RULEFIT

In [21]:
from skrules import SkopeRules
import metrics_rules
import time

In [22]:
# change dataset column names to x0, x1, x2, ...
X_train.columns = ['x_'+str(i) for i in range(len(X_train.columns))]
X_test.columns = ['x_'+str(i) for i in range(len(X_test.columns))]

In [23]:
clf = SkopeRules(max_depth_duplication=2,
                    n_estimators=512,
                    precision_min=0.3,
                    recall_min=0.1,
                    feature_names=X_train.columns.tolist())

In [24]:
start_time = time.time()
clf.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/skrules/skope_rules.py:220: UserWarning: Found labels {1, -1}. This method assumes target class to be labeled as 1 and normal data to be labeled as 0. Any label different from 0 will be considered as being from the target class.
  % set(self.classes_))


--- 13.085370779037476 seconds ---


In [25]:
start_time = time.time()
top_rules1 = clf.score_top_rules(X_test)
top_rules2 = clf.score_top_rules(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.06331872940063477 seconds ---


In [26]:
i = metrics_rules.calc_identity_rules(top_rules1, top_rules2)
print(i)

s = metrics_rules.calc_separability_rules(top_rules1)
print(s)

enc_rules = metrics_rules.exp_enc(clf, top_rules1)
sb = metrics_rules.calc_stability_rules(enc_rules, y_test)
print(sb)

(0.0, 593, 593)
(169398, 593, 351649, 48.17246743201317)
(72, 593)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [27]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_rules, X_test_norm)
print(sim)

0.0


# RULEMATRIX

In [42]:
categorical_features = X_train.columns.tolist()
continuous_features = X_train.columns.drop(categorical_features).tolist()

In [43]:
import rulematrix
import time
import metrics_rules

In [44]:
is_continuous = [True if i in continuous_features else False for i in X_train.columns.tolist()]
is_categorical = [True if i in categorical_features else False for i in X_train.columns.tolist()]

In [45]:
surrogate = rulematrix.surrogate.rule_surrogate(
    automl.predict,
    X_train,
    sampling_rate=4,
    is_continuous=is_continuous,
    is_categorical=is_categorical,
    seed=42
)

In [46]:
test_x = X_test.to_numpy()

In [47]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in range(len(xtest)):
        queried_rules = np.arange(surrogate.student.n_rules)[surrogate.student.decision_path(test_x[i].reshape(1,-1)).reshape(-1)]
        exp1.append(queried_rules[-1])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x))

In [48]:
start_time = time.time()
exp1 = exp_fn_blk(test_x)
exp2 = exp_fn_blk(test_x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.05279970169067383 seconds ---


In [49]:
def enc_exp(exp, n_features):
    enc = []
    for i in range(exp.shape[0]):
        new = np.zeros(n_features)
        for j in surrogate.student.rule_list[exp[i]].clauses:
            new[j.feature_idx] = 1
        enc.append(new)
    return np.array(enc)

In [50]:
enc_exp = enc_exp(exp1, X_train.shape[1])

In [51]:
i = metrics_rules.calc_identity_rules(exp1, exp2)
print(i)

s = metrics_rules.calc_separability_rules(exp1)
print(s)

sb = metrics_rules.calc_stability_rules(enc_exp, y_test)
print(sb)

(0.0, 593, 593)
(351056, 593, 351649, 99.83136593591905)
(0, 593)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)
/home/krish/interpretability/explanability/explanability/Model Training/metrics_rules.py:42: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  ct.fit(top_rules)


In [52]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_exp, X_test_norm)

In [53]:
sim

0.0